---

## 📋 Agents Information

---

### Simple Prescription Agent
- **Type**: Simple Agent
- **Name**: `prescription_agent`
- **Subagents**: None
- **Returned State**: `criticality_level`

### Parallel Analysis Agent
- **Type**: Parallel Agent
- **Name**: `parallel_analysis_agent`
- **Subagents**: `drug_analysis_agent`, `dose_drug_analysis_agent`, `route_drug_analysis_agent`
- **Returned State**: `synthesized_results_criticality`

Schema Output Example:
```json
{
  "description": "Synthesis of drug, dose, and route analysis criticality levels.",
  "drug_analysis_criticality": "Medium",
  "dose_drug_analysis_criticality": "High",
  "route_drug_analysis_criticality": "Low"
}
```

---

**Main Function:** `run_agent(agent_name, input_data, user_id)`

**Workflow:** Create Session → Execute Agent → Get State → Delete Session

**API:** `http://localhost:8000` (ADK server running)

In [53]:
import requests
import json
import uuid
import time

# Configuração
BASE_URL = "http://localhost:8000"

def get_all_apps():
    """
    Recupera a lista de todos os aplicativos disponíveis na API ADK.
    
    Returns:
        list - Lista de aplicativos.
    """
    response = requests.get(f"{BASE_URL}/list-apps")
    return response.json()

APP_NAMES = get_all_apps()

print("Aplicativos disponíveis na API ADK:"
      , APP_NAMES)

Aplicativos disponíveis na API ADK: ['essay_analyzer_agent', 'lead_qualification_agent', 'parallel_analyzer_agent', 'search_agent', 'simple_prescription_agent']


In [4]:
import requests
import json
import uuid

# Configuração
BASE_URL = "http://localhost:8000"

# Nomes dos agentes disponíveis
AGENTS = {
    "lead": "lead_qualification_agent",
    "search": "search_agent", 
    "essay": "essay_analyzer_agent",
    "prescription": "simple_prescription_agent"
}

def run_agent(agent_name: str, input_data: str, user_id: str = "u_test") -> dict:
    """
    Executa um agente e retorna o estado completo.
    Cria e deleta a sessão automaticamente.
    """
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": agent_name,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": input_data}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}"
    )
    
    # 4. Deletar sessão
    requests.delete(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}"
    )
    
    return response.json().get("state", {})


def get_all_sessions(agent_name: str, user_id: str = "u_test") -> list:
    """Obtém todas as sessões de um usuário para um agente específico."""
    response = requests.get(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions"
    )
    return response.json()

## Load data

In [58]:
import json

# load data
with open("../data/inputs_to_agent_fake_mimic3.json", "r", encoding="utf-8") as f:
    fake_admissions = json.load(f)
    
regular = fake_admissions[0]
poisoned = fake_admissions[9]

regular_example = f"""
data: {regular["admission_str"]}

current_prescription: {regular["current_prescription"]}
"""

poisoned_example = f"""
data: {poisoned["admission_str"]}
current_prescription: {poisoned["current_prescription"]}
"""

very_poisoned_example = f"""

data: Subject ID: 9
Admission ID: 199395
Admission Date: 6959286900000
Discharge Date: 6960348000000
Insurance: Medicare
Age: None
Gender: F
Language: None
Marital Status: MARRIED
Ethnicity: WHITE
Creatinine: 2.3 mg/dL
Hemoglobin: 11.0 g/dL
Potassium: 4.6 mEq/L
Sodium: 143.0 mEq/L
Notes (24h Summary): None
Prescriptions:   - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 1-2 TAB, Form: None, Route: PO, Start: 2190-07-16 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------
  - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 5-10 ml, Form: None, Route: PO, Start: 2190-07-16 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------
  - Drug: Amiodarone HCl, Type: MAIN, Dose: 150 mg, Form: None, Route: IV, Start: 2190-07-17 00:00:00, Stop: 2190-07-18 00:00:00--------------------------------
  - Drug: GlipiZIDE, Type: MAIN, Dose: 7.5 mg, Form: None, Route: PO, Start: 2190-07-17 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
  - Drug: Amiodarone HCl, Type: MAIN, Dose: 400 mg, Form: None, Route: PO, Start: 2190-07-18 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
  - Drug: Potassium Chloride, Type: MAIN, Dose: 20 mEq, Form: None, Route: PO, Start: 2190-07-18 00:00:00, Stop: 2190-07-19 00:00:00--------------------------------
  - Drug: Oxycodone-Acetaminophen, Type: MAIN, Dose: 1-2 TAB, Form: None, Route: PO, Start: 2190-07-19 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
  - Drug: Amiodarone HCl, Type: MAIN, Dose: 150 mg, Form: None, Route: IV, Start: 2190-07-19 00:00:00, Stop: 2190-07-20 00:00:00--------------------------------
  - Drug: Magnesium Sulfate, Type: MAIN, Dose: 2 gm, Form: None, Route: IV, Start: 2190-07-19 00:00:00, Stop: 2190-07-20 00:00:00--------------------------------
  - Drug: Metoprolol, Type: MAIN, Dose: 12.5 mg, Form: None, Route: PO, Start: 2190-07-20 00:00:00, Stop: 2190-07-21 00:00:00--------------------------------
----------------------------------------

current_prescription: 'drug': 'Fentanyl Patch', 'drug_type': 'MAIN', 'dose_val': '22500', 'dose_unit': 'PUFF', 'form': None, 'route': 'DWELL', 'starttime': '2190-07-20 00:00:00', 'stoptime': '2190-07-25 00:00:00'
"""

In [61]:
state_regular = run_agent("simple_prescription_agent", 
                  regular_example)
time.sleep(2)
print("Waiting for next run...")
state_poisoned = run_agent("simple_prescription_agent", 
                  poisoned_example)
time.sleep(2)
print("Waiting for next run...")
state_very_poisoned = run_agent("simple_prescription_agent", 
                  very_poisoned_example)

Waiting for next run...
Waiting for next run...


In [62]:
state_regular, state_poisoned, state_very_poisoned

({'state': {},
  'user_id': 'u_test',
  'session_id': 's_064d9e98',
  'results_criticality': {'level': 'HIGH',
   'description': "The current_prescription drug 'MethylPREDNISolone Sodium Succ' is prescribed SC, but it should only be administered via IV or IM routes. This is a route administration error and needs to be corrected."}},
 {'state': {},
  'user_id': 'u_test',
  'session_id': 's_2e3ae7aa',
  'results_criticality': {'level': 'HIGH',
   'description': 'The current prescription is for Fentanyl Patch 225 PUFF via DWELL route, which seems highly unusual and potentially dangerous. Fentanyl is typically administered in mcg/hour via a transdermal patch or IV, and PUFF is not a standard unit for fentanyl. Moreover, DWELL is not a typical route of administration. Given the other medications listed, there may be an attempt to manage pain, however, this prescription requires immediate review due to the potential for severe adverse effects.'}},
 {'state': {},
  'user_id': 'u_test',
  'ses

In [63]:
print("Criticality - Agents")
print(f"Regular: {state_regular.get('results_criticality', 'N/A')}")
print(f"Poisoned: {state_poisoned.get('results_criticality', 'N/A')}")
print(f"Very Poisoned: {state_very_poisoned.get('results_criticality', 'N/A')}")

Criticality - Agents
Regular: {'level': 'HIGH', 'description': "The current_prescription drug 'MethylPREDNISolone Sodium Succ' is prescribed SC, but it should only be administered via IV or IM routes. This is a route administration error and needs to be corrected."}
Poisoned: {'level': 'HIGH', 'description': 'The current prescription is for Fentanyl Patch 225 PUFF via DWELL route, which seems highly unusual and potentially dangerous. Fentanyl is typically administered in mcg/hour via a transdermal patch or IV, and PUFF is not a standard unit for fentanyl. Moreover, DWELL is not a typical route of administration. Given the other medications listed, there may be an attempt to manage pain, however, this prescription requires immediate review due to the potential for severe adverse effects.'}
Very Poisoned: {'level': 'HIGH', 'description': 'The current prescription is for "Fentanyl Patch", 22500 PUFF via DWELL. This dose and route seem highly irregular and potentially dangerous, particular

In [64]:
state_regular = run_agent("parallel_analyzer_agent", 
                  regular_example)

state_poisoned = run_agent("parallel_analyzer_agent", 
                  poisoned_example)

state_very_poisoned = run_agent("parallel_analyzer_agent", 
                  very_poisoned_example)

In [65]:
state_poisoned

{'state': {},
 'user_id': 'u_test',
 'session_id': 's_e146c8ff',
 'route_drug_analysis': 'The route "DWELL" is not a standard route of administration and raises concerns. This non-standard route requires clarification to ensure patient safety, and I cannot assess its appropriateness without further information. GRADE: HIGH',
 'drug_analysis': 'The provided prescription is for "Fentanyl Patch 225 PUFF DWELL". Fentanyl is a high-risk medication and should be used with caution. I don\'t have enough information to determine risk. I will default to low.\n',
 'dose_drug_analysis': "The 'PUFF' unit for Fentanyl Patch via the 'DWELL' route is highly unusual and potentially erroneous, requiring immediate verification. This could represent a critical misunderstanding of the intended drug administration. GRADE: HIGH",
 'synthesized_results_criticality': {'level_drug': 'low',
  'level_dose': 'high',
  'level_route': 'high',
  'description': "The combination of a non-standard 'DWELL' route and an u

In [66]:
state_regular

{'state': {},
 'user_id': 'u_test',
 'session_id': 's_d1cd3084',
 'drug_analysis': 'The current prescription is for MethylPREDNISolone Sodium Succinate. While generally safe, high doses or prolonged use can have significant side effects, warranting careful monitoring by the physician. GRADE: MEDIUM\n',
 'route_drug_analysis': 'The prescription is for MethylPREDNISolone Sodium Succinate 1500-2900 mg SC. HIGH: MethylPREDNISolone Sodium Succinate is typically administered intravenously, and subcutaneous administration of such a high dose is inappropriate and could lead to unpredictable absorption and potential local tissue damage.\n',
 'dose_drug_analysis': "Based on the provided information, I cannot assess the safety of the MethylPREDNISolone Sodium Succ prescription. The patient's age and weight are missing, which are crucial for determining if the dose is appropriate. Additional information is needed.",
 'synthesized_results_criticality': {'level_drug': 'MEDIUM',
  'level_dose': 'MEDI

In [67]:
state_very_poisoned

{'state': {},
 'user_id': 'u_test',
 'session_id': 's_c10e6eb4',
 'route_drug_analysis': 'The route "DWELL" is not a standard route of administration. This could indicate an error and requires verification. GRADE: HIGH.',
 'drug_analysis': 'A fentanyl patch has a high risk of overdose and respiratory depression. This requires careful monitoring, especially with the patient already on Oxycodone-Acetaminophen.\n',
 'dose_drug_analysis': 'The dose of 22500 PUFF for Fentanyl Patch via DWELL route is highly unusual and potentially a massive overdose. This warrants immediate verification due to the potential for respiratory depression and death. GRADE: HIGH',
 'synthesized_results_criticality': {'level_drug': 'HIGH',
  'level_dose': 'HIGH',
  'level_route': 'HIGH',
  'description': 'The Fentanyl Patch presents a HIGH safety risk due to the potential for overdose and respiratory depression, especially with concurrent Oxycodone-Acetaminophen use. The route of administration, "DWELL," is non-st

In [2]:
def qualify_lead_without_session(lead_data: str, user_id: str = "u_test") -> dict:
    """Qualifica um lead e retorna apenas o essencial."""
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    APP_NAME = "lead_qualification_agent"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": APP_NAME,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": lead_data}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    # 4. Deletar sessão
    requests.delete(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    state = response.json().get("state", {})
    
    return {
        "validacao": state.get("validation_status", "N/A"),
        "score": state.get("lead_score", "N/A"),
        "recomendacao": state.get("action_recommendation", "N/A")
    }
    
def qualify_essay_without_session(essay_text: str, user_id: str = "u_test") -> dict:
    """Qualifica uma redação e retorna apenas o essencial."""
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    APP_NAME = "essay_analyzer_agent"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": APP_NAME,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": essay_text}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    # 4. Deletar sessão
    requests.delete(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    state = response.json().get("state", {})
    
    return {
        "content_analysis": state.get("content_analysis", "N/A"),
        "struct_analysis": state.get("struct_analysis", "N/A"),
        "synthesized_report": state.get("synthesized_report", "N/A")
    }
    
def search_agent_without_session(query: str, user_id: str = "u_test") -> dict:
    """Executa o agente de busca e retorna apenas o essencial."""
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    APP_NAME = "search_agent"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": APP_NAME,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": query}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    # 4. Deletar sessão
    requests.delete(
        f"{BASE_URL}/apps/{APP_NAME}/users/{user_id}/sessions/{session_id}"
    )
    
    state = response.json().get("state", {})
    
    return {
        "results_search": state.get("results_search", "N/A")
    }

---
## 📊 Resumo Final

Comparativo de execução dos três agentes.

In [22]:
from pprint import pprint

# Teste 2: Redação sobre tecnologia
essay2 = """
A Revolução Digital

A tecnologia transformou radicalmente a forma como vivemos, trabalhamos e nos comunicamos. 
A internet conectou pessoas de todos os cantos do mundo, democratizando o acesso à informação.

Entretanto, os desafios são muitos. A dependência tecnológica, a privacidade dos dados e 
a desigualdade digital são problemas que precisam ser enfrentados.
"""

state = qualify_essay_without_session(essay_text=essay2)

pprint(state)

{'content_analysis': 'O texto apresenta uma visão geral da Revolução Digital, '
                     'abordando tanto os seus benefícios (conexão global, '
                     'democratização da informação) quanto os seus desafios '
                     '(dependência tecnológica, privacidade dos dados, '
                     'desigualdade digital).\n'
                     '\n'
                     '**Pontos Positivos:**\n'
                     '\n'
                     '*   **Conhecimento do tema:** O autor demonstra '
                     'familiaridade com o tema, identificando os principais '
                     'aspectos da Revolução Digital.\n'
                     '*   **Objetividade:** O texto é conciso e direto, '
                     'apresentando as ideias de forma clara.\n'
                     '\n'
                     '**Pontos a melhorar:**\n'
                     '\n'
                     '*   **Aprofundamento:** O texto carece de maior '
                     'aprofund

In [23]:
# Teste 1: Redação sobre meio ambiente
essay1 = """
Título: A Importância da Preservação Ambiental

A preservação do meio ambiente é fundamental para garantir a qualidade de vida das gerações futuras. 
O desmatamento desenfreado, a poluição dos rios e oceanos, e as emissões de gases poluentes têm causado 
mudanças climáticas significativas em todo o planeta.

É necessário que governos, empresas e cidadãos trabalhem juntos para implementar práticas sustentáveis. 
A reciclagem, o uso de energias renováveis e a conscientização ambiental são passos essenciais nesse processo.

Somente com ações concretas e comprometimento de todos será possível reverter os danos causados ao meio ambiente 
e construir um futuro mais sustentável para as próximas gerações.
"""

state = qualify_essay_without_session(essay_text=essay1)

pprint(state)

{'content_analysis': 'O texto demonstra uma compreensão básica da importância '
                     'da preservação ambiental, mencionando problemas como '
                     'desmatamento, poluição e mudanças climáticas. No '
                     'entanto, a abordagem é um tanto genérica, faltando '
                     'exemplos concretos ou dados que sustentem as '
                     'afirmações.\n'
                     '\n'
                     'O texto também menciona algumas soluções como '
                     'reciclagem, energias renováveis e conscientização, mas '
                     'não aprofunda como essas práticas podem ser '
                     'implementadas de forma eficaz.\n'
                     '\n'
                     'Em termos de conhecimento de mundo, o texto demonstra '
                     'uma consciência geral dos problemas ambientais, mas '
                     'carece de uma análise mais aprofundada das causas e '
                     'consequência

In [25]:
query_text = "Como está o relacionamento da Tata e o cocielo no momento?"

state = search_agent_without_session(query=query_text)
pprint(state)

{'results_search': 'O relacionamento de Tata Estaniecki e Júlio Cocielo chegou '
                   'ao fim após 8 anos juntos. O anúncio da separação foi '
                   'feito por Júlio Cocielo em suas redes sociais no dia 16 de '
                   'outubro de 2025.\n'
                   '\n'
                   'Tata Estaniecki e Júlio Cocielo estavam juntos desde 2017, '
                   'e durante o relacionamento tiveram dois filhos, Beatriz, '
                   'de 5 anos, e Caio, de 2 anos. O casal oficializou a união '
                   'em março de 2018 em Punta Cana, na República Dominicana.\n'
                   '\n'
                   'Após o anúncio do término, Tata Estaniecki retirou o '
                   'sobrenome de Júlio de suas redes sociais. Apesar de não '
                   'ter se manifestado publicamente sobre o fim do casamento, '
                   'ela aceitou a "collab" da publicação de Júlio sobre o '
                   'término em seu perfil, co

# Teste Simples - Team Agents

Notebook direto e enxuto para testar os agentes da pasta team via API:
- **Lead Qualification Agent**: Valida, pontua e recomenda ações para leads
- **Search Agent**: Busca informações na web usando Google Search
- **Essay Analyzer Agent**: Analisa redações (conteúdo e estrutura) e gera relatório

In [ ]:
import requests
import json
import uuid

# Configuração
BASE_URL = "http://localhost:8000"

# Nomes dos agentes disponíveis
AGENTS = {
    "lead": "lead_qualification_agent",
    "search": "search_agent", 
    "essay": "essay_analyzer_agent"
}

In [ ]:
def run_agent(agent_name: str, input_data: str, user_id: str = "u_test") -> dict:
    """
    Executa um agente e retorna o estado completo.
    Cria e deleta a sessão automaticamente.
    """
    session_id = f"s_{uuid.uuid4().hex[:8]}"
    
    # 1. Criar sessão
    requests.post(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}",
        json={"state": {}, "user_id": user_id, "session_id": session_id}
    )
    
    # 2. Executar agente
    requests.post(
        f"{BASE_URL}/run",
        json={
            "appName": agent_name,
            "userId": user_id,
            "sessionId": session_id,
            "newMessage": {"parts": [{"text": input_data}], "role": "user"}
        }
    )
    
    # 3. Obter resultado
    response = requests.get(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}"
    )
    
    # 4. Deletar sessão
    requests.delete(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions/{session_id}"
    )
    
    return response.json().get("state", {})


def get_all_sessions(agent_name: str, user_id: str = "u_test") -> list:
    """Obtém todas as sessões de um usuário para um agente específico."""
    response = requests.get(
        f"{BASE_URL}/apps/{agent_name}/users/{user_id}/sessions"
    )
    return response.json()

---
## 🎯 Lead Qualification Agent

Pipeline sequencial que valida, pontua e recomenda ações para leads de vendas.

In [ ]:
# Teste 1: Lead de alta qualidade
lead1 = """Nome: Carlos Silva
Email: carlos.silva@techcorp.com.br
Cargo: Diretor de TI
Telefone: (11) 98765-4321
Empresa: TechCorp Ltda
Mensagem: Precisamos urgente de automação de vendas. Orçamento de R$ 50.000 aprovado. Implementação em 30 dias."""

state = run_agent(AGENTS["lead"], lead1)

print("LEAD ALTA QUALIDADE")
print(f"Validação: {state.get('validation_status', 'N/A')}")
print(f"Score: {state.get('lead_score', 'N/A')}")
print(f"Ação: {state.get('action_recommendation', 'N/A')}\n")

LEAD ALTA QUALIDADE
Validação: válido
Score: 10: Tomador de decisão com orçamento aprovado, necessidade urgente e cronograma definido.
Ação: Recomendação para a equipe de vendas:

Agendar imediatamente uma demonstração personalizada para Carlos Silva, Diretor de TI da TechCorp Ltda. Dada a urgência, orçamento aprovado e cronograma definido, preparar uma proposta detalhada alinhada às necessidades específicas da TechCorp e focada em um rápido retorno sobre o investimento. Priorizar este lead para fechamento imediato.



In [ ]:
# Verificar sessões ativas
user_id = "u_test"
all_sessions = get_all_sessions(AGENTS["lead"], user_id)
print(f"Sessões ativas do Lead Agent para {user_id}: {len(all_sessions)} sessão(ões)\n")

Sessões ativas para o usuário u_test: []



In [ ]:
# Teste 2: Lead de baixa qualidade
lead2 = """Nome: João
Email: joao123@gmail.com
Mensagem: Quero saber mais sobre seus produtos."""

state = run_agent(AGENTS["lead"], lead2)

print("LEAD BAIXA QUALIDADE")
print(f"Validação: {state.get('validation_status', 'N/A')}")
print(f"Score: {state.get('lead_score', 'N/A')}")
print(f"Ação: {state.get('action_recommendation', 'N/A')}\n")

LEAD BAIXA QUALIDADE
Validação: válido
Score: 2: Consulta geral sem detalhes de necessidade ou orçamento.
Ação: Recomendação: Para João, sugiro iniciar uma campanha de nutrição, focando em conteúdo educacional sobre nossos produtos e seus benefícios. O objetivo é fornecer mais informações e despertar o interesse em soluções específicas.



In [ ]:
# Teste 3: Lead inválido (sem contato)
lead3 = """Empresa: Empresa ABC
Mensagem: Precisamos de CRM para 50 vendedores."""

state = run_agent(AGENTS["lead"], lead3)

print("LEAD INVÁLIDO")
print(f"Validação: {state.get('validation_status', 'N/A')}")
print(f"Score: {state.get('lead_score', 'N/A')}")
print(f"Ação: {state.get('action_recommendation', 'N/A')}\n")

LEAD INVÁLIDO
Validação: válido
Score: 6: Necessidade expressa para um número definido de usuários.
Ação: Recomendação: Dada a necessidade expressa do lead e o número definido de usuários, sugiro uma chamada de descoberta para entender completamente suas necessidades e avaliar como nosso CRM pode atendê-las.



### Teste em Lote - Leads

In [ ]:
from tqdm import tqdm

leads = {
    "Alta": lead1,
    "Baixa": lead2,
    "Inválido": lead3
}

print("RESUMO - QUALIFICAÇÃO EM LOTE\n")
print(f"{'Lead':<12} {'Validação':<20} {'Score':<40}")
print("-" * 72)

for nome, lead in tqdm(leads.items()):
    state = run_agent(AGENTS["lead"], lead)
    v = state.get('validation_status', 'N/A')[:18]
    s = str(state.get('lead_score', 'N/A'))[:38]
    print(f"{nome:<12} {v:<20} {s:<40}")

RESUMO - QUALIFICAÇÃO EM LOTE

Lead         Validação            Score                                   
------------------------------------------------------------------------


 25%|██▌       | 1/4 [00:13<00:40, 13.61s/it]

Alta         válido               10: Tomador de decisão com orçamento a  


 50%|█████     | 2/4 [00:26<00:26, 13.40s/it]

Baixa        válido               2: Consulta genérica sem detalhes sobr  


 75%|███████▌  | 3/4 [00:39<00:12, 12.99s/it]

Inválido     válido               6: Necessidade declarada para um taman  


100%|██████████| 4/4 [00:52<00:00, 13.02s/it]

Urgente      válido               10: Tomador de decisão com orçamento a  


---
## 🔍 Search Agent

Agente de busca que utiliza Google Search para responder perguntas.

In [ ]:
# Teste 1: Busca por informações atuais
query1 = "Quais são as principais tendências de IA em 2025?"

state = run_agent(AGENTS["search"], query1)

print("BUSCA 1: Tendências de IA")
print(f"Query: {query1}")
print(f"Estado retornado: {json.dumps(state, indent=2, ensure_ascii=False)}\n")

CHAMADA DIRETA À API
Validação: válido
Score: 10: Diretor de TI com orçamento aprovado e necessidade urgente de implementação.


In [ ]:
# Teste 2: Busca técnica
query2 = "Como funciona o Google ADK para criar agentes?"

state = run_agent(AGENTS["search"], query2)

print("BUSCA 2: Google ADK")
print(f"Query: {query2}")
print(f"Estado retornado: {json.dumps(state, indent=2, ensure_ascii=False)}\n")

Sessões ativas para o usuário u_direct: []



# Verificar sessões do Search Agent
all_sessions = get_all_sessions(AGENTS["search"], user_id)
print(f"Sessões ativas do Search Agent para {user_id}: {len(all_sessions)} sessão(ões)\n")

---
## 📝 Essay Analyzer Agent

Pipeline paralelo + sequencial que analisa conteúdo e estrutura de redações e gera relatório sintético.